In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# Maker Functionality [NB20]


In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=False, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
sdkaddr = SDK.addr()
print(sdkaddr)
SDK.version().get("msg")

0x02aA0c42702110747c4cEa3C4Daf4D6cF701a6Ee


'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(index=1, options=(('USDC/USDT', 'USDC/USDT'), ('USDC/ETH', 'USDC/ETH'), ('BNT/USDT', 'BNT/USDT'), ('B…

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [6]:
m = SDK.mGetMarketByPair(pair=pair)
m

MarketByPair(pair='ETH/USDC', bestBid=1499.999999795006, bestAsk=1630.0000000000148, liqBid=2497052.640732, liqAsk=2154.036407671459, lastBid=799.9999998679182, lastAsk=3000.0000000000327)

In [7]:
SDK.roundsd(m.lastBid,6), SDK.roundsd(m.bestBid,6), SDK.roundsd(m.mid,6), SDK.roundsd(m.bestAsk,6), SDK.roundsd(m.lastAsk,6)

(800.0, 1500.0, 1565.0, 1630.0, 3000.0)

In [8]:
(SDK.roundsd(m.liqBid,4), m.liqBidUnit, SDK.roundsd(m.liqAsk,4), m.liqAskUnit)

(2497100.0, 'USDC', 2154.0, 'ETH')

## Create strategy

#### `mCreateStrategy`

In [9]:
try:
    strat_w()
except:
    strat_w = PcSliderManager(choices=["buyStart", "buyEnd", "sellStart", "sellEnd"], values=[0, 1, 0, 1])
    strat_w()

In [10]:
sellAmountTknB =     5
buyAmountTknQ  = 10000
x = strat_w.values
print(f"pair = {pair}, TknB={tknb}, TknQ={tknq}")
print(f"buy  {tknb} {SDK.roundsd(m.ibid(x[0]),6)} down to {SDK.roundsd(m.ibid(x[1]),6)}   [{sellAmountTknB} {tknq}] ")
print(f"sell {tknb} {SDK.roundsd(m.iask(x[2]),6)}      to {SDK.roundsd(m.iask(x[3]),6)}   [{buyAmountTknQ} {tknb}] ")

pair = ETH/USDC, TknB=ETH, TknQ=USDC
buy  ETH 1500.0 down to 800.0   [5 USDC] 
sell ETH 1630.0      to 3000.0   [10000 ETH] 


In [11]:
x = strat_w.values
tx = SDK.mCreateStrategy(
    pair, 
    buyRangeStart  = m.ibid(x[0]), buyRangeEnd  = m.ibid(x[1]), buyAmountTknQ = buyAmountTknQ, 
    sellRangeStart = m.iask(x[2]), sellRangeEnd = m.iask(x[3]), sellAmountTknB = sellAmountTknB, 
    overrides = None)
tx

{'data': '0xf727473a000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb480000000000000000000000000000000000000000000000004563918244f400000000000000000000000000000000000000000000000000004563918244f40000000000000000000000000000000000000000000000000000000dcb7b9cf94fa0000000000000000000000000000000000000000000000000000f8ea2d65093c400000000000000000000000000000000000000000000000000000002540be40000000000000000000000000000000000000000000000000000000002540be40000000000000000000000000000000000000000000000000000000000af3f591c00000000000000000000000000000000000000000000000000000001da88051e',
 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x4563918244f40000'}}

In [12]:
try:
    submit_w()
except:
    submit_w = CheckboxManager(["submit"], values=[0])
    submit_w()

In [13]:
if submit_w.values[0]: 
    r = SDK.signsubmittx(tx, sign=True)
else:
    r = "check box above to submit"
r

'check box above to submit'

## Review strategies

In [14]:
data = SDK.mGetUserStrategies(user=sdkaddr)
data[0]

Strategy(sid=492, baseToken='BNT', quoteToken='DAI', buyPriceLow=0.2, buyPriceHigh=0.2, buyBudget=100.0, sellPriceLow=0.4, sellPriceHigh=0.4, sellBudget=200.0, encoded=EncodedStrategy(sid=492, order0=EncodedOrder(token='BNT', y=200000000000000000000, z=200000000000000000000, A=0, B=504000484397786), order1=EncodedOrder(token='DAI', y=100000000000000000000, z=100000000000000000000, A=0, B=125879436378039)))

In [15]:
d, de=data[0], data[0].encoded
print(de.order1.descr)
print(de.descr[1])
print(d.descr[1])
f"buying {d.baseToken} @", de.order1.p_start, de.order1.p_end, f"{d.quoteToken} per {d.baseToken}"

selling DAI @ (5.000000000000031..5.000000000000031)  [TKNwei] per DAIwei
order1: buying  BNT @ (0.19999999999999876..0.19999999999999876) DAIwei per BNTwei
order1: buying  BNT @ (0.2..0.2) DAI per BNT


('buying BNT @', 0.19999999999999876, 0.19999999999999876, 'DAI per BNT')

In [16]:
d, de=data[0], data[0].encoded
print(de.order0.descr)
print(de.descr[0])
print(d.descr[0])
f"selling {d.baseToken} @", 1/de.order0.p_start, 1/de.order0.p_end, f"{d.quoteToken} per {d.baseToken}"

selling BNT @ (0.4000000000000021..0.4000000000000021)  [TKNwei] per BNTwei
order0: selling BNT @ (0.4000000000000021..0.4000000000000021) DAIwei per BNTwei
order0: selling BNT @ (0.4..0.4) DAI per BNT


('selling BNT @', 0.4000000000000021, 0.4000000000000021, 'DAI per BNT')

In [21]:
print(f"Number of strategies: {len(data)}")

Number of strategies: 1


In [22]:
Sim = CarbonSimulatorUI(verbose=False, raiseonerror=True)
for d in data:
    Sim.add_fromsdk(d, 6)
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid,cid
0,0,BNTDAI,BNT,200.0,200.0,BNT,False,0.4,0.4,0.4,DAI per BNT,1,492
1,1,BNTDAI,DAI,100.0,100.0,DAI,False,0.2,0.2,0.2,DAI per BNT,0,492


## Delete strategies

In [23]:
stratids = [d["id"] for d in data]
try:
    delete_w()
except:
    delete_w = CheckboxManager(stratids)
    delete_w()

In [24]:
try:
    submitd_w()
except:
    submitd_w = CheckboxManager(["submit delete"], values=[0])
    submitd_w()

In [25]:
for cid, ds in zip(stratids, delete_w.values):
    if ds:
        data = SDK.deleteStrategy(strategyId=cid)
        #print(data)
        if submitd_w.values[0]:
            result = SDK.signsubmittx(data, sign=True)
            print(f"Deleteting strategy {cid}: ", result)
            delete_w = None
        else:
            print(f"Would delete strategy {cid} if box was ticked ")